In [11]:
from language_learning_tools.grouping.vocab_grouping import \
    get_hangul_parts, \
    group_lang_df_by_parts, \
    get_hangul_hanja_tuples
import pandas as pd

INPUT_CSV_PATH = '../input_generated_files/english_hangul_hanja_tups_to_memorize.csv'
OUTPUT_CSV_PATH = '../output/tups_grouped_by_hangul_hanja.csv'
ENGLISH_COL_NAME = 'English_translation'
KOREAN_COL_NAME = 'hangul_word'
HANJA_COL_NAME = 'hanja_word_with_x'

HANJA_HANGUL_TUPLE = 'hangul_hanja_tuple'
HANJA_HANGUL_CHAR_TUPLE = 'hangul_hanja_char_tuple'

HANJA_CHAR_COL_NAME = 'hanja_char'

HANJA_PARTS_COL_NAME = 'hanja_parts'

eng_kor_df = pd.read_csv(INPUT_CSV_PATH)

eng_kor_df[HANJA_HANGUL_TUPLE] = eng_kor_df.apply(lambda row: (row[KOREAN_COL_NAME], row[HANJA_COL_NAME]), axis=1)

hanja_chars_output_df = group_lang_df_by_parts(
    eng_kor_df,
    HANJA_HANGUL_TUPLE,
    get_hangul_hanja_tuples,
    HANJA_HANGUL_CHAR_TUPLE,
    [KOREAN_COL_NAME, HANJA_COL_NAME, ENGLISH_COL_NAME])

hanja_chars_output_df['hangul_char'] = hanja_chars_output_df['hangul_hanja_char_tuple'].apply(lambda x: x[0])
hanja_chars_output_df['hanja_char'] = hanja_chars_output_df['hangul_hanja_char_tuple'].apply(lambda x: x[1])

hangul_hanja_out_df = hanja_chars_output_df[
    ['hangul_char',
     'hanja_char',
     KOREAN_COL_NAME,
     HANJA_COL_NAME,
     ENGLISH_COL_NAME]
]

/Users/zacharydestefano/git/language_learning/src/language_learning_tools/grouping/vocab_grouping.py:127: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  parts_in_multiple_words[num_parts_col_name].apply(lambda x: int(x))


In [23]:
hangul_rank = hangul_hanja_out_df[['hangul_char']]\
    .drop_duplicates()\
    .reset_index()\
    .reset_index()[['level_0', 'hangul_char']].rename(columns={
        'level_0': 'hangul_rank'
    }).set_index('hangul_char')

hanja_rank = hangul_hanja_out_df[['hanja_char']]\
    .drop_duplicates()\
    .reset_index()\
    .reset_index()[['level_0', 'hanja_char']].rename(columns={
        'level_0': 'hanja_rank'
    }).set_index('hanja_char')

hangul_hanja_out_df_with_rank = hangul_hanja_out_df\
    .join(hangul_rank, on='hangul_char')\
    .join(hanja_rank, on='hanja_char')

In [24]:
hangul_hanja_out_df_with_rank

,hangul_char,hanja_char,hangul_word,hanja_word_with_x,English_translation,hangul_rank,hanja_rank
72,아,X,할아버지,X,grandfather,0,0
88,아,X,피아노,X,piano,0,0
168,아,X,좋아하다,X,to like,0,0
286,아,X,아홉,X,nine,0,0
287,아,X,아프리카,X,Africa,0,0
...,...,...,...,...,...,...,...
543,결,結,결혼,結婚,marriage,171,79
154,개,X,지우개,X,eraser,172,0
548,개,X,개,X,counting unit for general items,172,0
317,간,間,시간표,時間表,time table,173,80


In [ ]:
hangul_hanja_out_df.to_csv(OUTPUT_CSV_PATH, index=False)

